# AWS SageMaker Model Store: Time Series Demand Forecasting

This notebook creates a model store for the time series forecasting models trained in `Modeling.ipynb`.

It demonstrates:
1. **Model Package Group** - Container for versioned models
2. **Model Packages** - Individual model versions with inference specifications
3. **Model Cards** - Metadata and documentation for each model
4. **Model Registry** - Track trained models across versions

## Setup & Initialize SageMaker

In [ ]:
import boto3
import sagemaker
import pandas as pd
import pickle
import json
import time
from datetime import datetime
from io import BytesIO

# Initialize SageMaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
s3_client = boto3.client('s3', region_name=region)
sm_client = boto3.client('sagemaker', region_name=region)

print(f"SageMaker Role: {role}")
print(f"Default S3 Bucket: {bucket}")
print(f"Region: {region}")

## Part 1: Load & Serialize Trained Models


In [ ]:
# Load the data and re-train models (or load from disk if available)
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

# Load and prepare data
df = pd.read_csv("dataset.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_agg = df.groupby('timestamp')['value'].sum().reset_index()
df_agg.columns = ['ds', 'y']
df_agg = df_agg.sort_values('ds').reset_index(drop=True)

print(f"Data shape: {df_agg.shape}")
print(f"Date range: {df_agg['ds'].min()} to {df_agg['ds'].max()}")

In [ ]:
# Feature engineering function
def create_features(df):
    df = df.copy()
    df['month']         = df['ds'].dt.month
    df['quarter']       = df['ds'].dt.quarter
    df['year']          = df['ds'].dt.year
    df['month_sin']     = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos']     = np.cos(2 * np.pi * df['month'] / 12)
    df['trend']         = np.arange(len(df))

    for lag in [1, 2, 3, 6, 12]:
        df[f'lag_{lag}'] = df['y'].shift(lag)

    df['rolling_mean_3']  = df['y'].shift(1).rolling(3).mean()
    df['rolling_mean_12'] = df['y'].shift(1).rolling(12).mean()
    df['rolling_std_3']   = df['y'].shift(1).rolling(3).std()

    return df

df_feat = create_features(df_agg)
df_feat = df_feat.dropna().reset_index(drop=True)

print(f"Feature engineered data shape: {df_feat.shape}")

In [ ]:
# Train/Test split
TEST_MONTHS = 12

feature_cols = [c for c in df_feat.columns if c not in ['ds', 'y']]

train = df_feat.iloc[:-TEST_MONTHS]
test  = df_feat.iloc[-TEST_MONTHS:]

X_train, y_train = train[feature_cols], train['y']
X_test,  y_test  = test[feature_cols],  test['y']

train_ts = df_agg.iloc[:-TEST_MONTHS]['y']
test_ts  = df_agg.iloc[-TEST_MONTHS:]['y']

print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")
print(f"Features: {len(feature_cols)}")

In [ ]:
# Train all models
models_dict = {}
results_list = []

def evaluate(name, y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((np.array(y_true) - np.array(y_pred)) / np.array(y_true))) * 100
    return {'Model': name, 'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}

# 1. Linear Regression
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_sc, y_train)
lr_pred = lr.predict(X_test_sc)
models_dict['Linear Regression'] = (lr, scaler, 'StandardScaler')
results_list.append(evaluate("Linear Regression", y_test, lr_pred))
print("✓ Linear Regression trained")

# 2. Ridge Regression
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_sc, y_train)
ridge_pred = ridge.predict(X_test_sc)
models_dict['Ridge Regression'] = (ridge, scaler, 'StandardScaler')
results_list.append(evaluate("Ridge Regression", y_test, ridge_pred))
print("✓ Ridge Regression trained")

# 3. Random Forest
rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
models_dict['Random Forest'] = (rf, None, None)
results_list.append(evaluate("Random Forest", y_test, rf_pred))
print("✓ Random Forest trained")

# 4. XGBoost
xgb = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, 
                   subsample=0.8, colsample_bytree=0.8, random_state=42, verbosity=0)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
models_dict['XGBoost'] = (xgb, None, None)
results_list.append(evaluate("XGBoost", y_test, xgb_pred))
print("✓ XGBoost trained")

# 5. LightGBM
lgbm = LGBMRegressor(n_estimators=300, learning_rate=0.05, max_depth=4,
                     subsample=0.8, colsample_bytree=0.8, random_state=42, verbose=-1)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_test)
models_dict['LightGBM'] = (lgbm, None, None)
results_list.append(evaluate("LightGBM", y_test, lgbm_pred))
print("✓ LightGBM trained")

# 6. ETS (Exponential Smoothing)
ets_model = ExponentialSmoothing(train_ts, trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)
ets_pred = ets_model.forecast(TEST_MONTHS)
models_dict['ETS'] = (ets_model, None, None)
results_list.append(evaluate("ETS (Holt-Winters)", test_ts.values, ets_pred.values))
print("✓ ETS (Holt-Winters) trained")

# 7. SARIMA
sarima_model = SARIMAX(train_ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12),
                        enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
sarima_pred = sarima_model.forecast(TEST_MONTHS)
models_dict['SARIMA'] = (sarima_model, None, None)
results_list.append(evaluate("SARIMA(1,1,1)(1,1,1,12)", test_ts, sarima_pred))
print("✓ SARIMA trained")

In [ ]:
# Compare models
results_df = pd.DataFrame(results_list).sort_values('RMSE').reset_index(drop=True)
print("\n=== Model Performance (sorted by RMSE) ===")
print(results_df.to_string(index=False))

best_model_name = results_df.iloc[0]['Model']
print(f"\n🏆 Best Model: {best_model_name}")

In [ ]:
# Save models to S3
s3_prefix = 'timeseries-demand-forecasting/models'

model_metadata = {}

for model_name, (model_obj, scaler_obj, scaler_type) in models_dict.items():
    # Serialize model
    model_bytes = pickle.dumps(model_obj)
    
    # Save to S3
    s3_key = f"{s3_prefix}/{model_name.replace(' ', '_').replace('(', '').replace(')', '')}.pkl"
    s3_client.put_object(Bucket=bucket, Key=s3_key, Body=model_bytes)
    
    # Save scaler if exists
    if scaler_obj is not None:
        scaler_bytes = pickle.dumps(scaler_obj)
        scaler_key = f"{s3_prefix}/{model_name.replace(' ', '_').replace('(', '').replace(')', '')}_scaler.pkl"
        s3_client.put_object(Bucket=bucket, Key=scaler_key, Body=scaler_bytes)
        model_metadata[model_name] = {
            'model_s3_path': f"s3://{bucket}/{s3_key}",
            'scaler_s3_path': f"s3://{bucket}/{scaler_key}",
            'scaler_type': scaler_type
        }
    else:
        model_metadata[model_name] = {
            'model_s3_path': f"s3://{bucket}/{s3_key}",
            'scaler_s3_path': None,
            'scaler_type': None
        }

print(f"✓ All models saved to S3: s3://{bucket}/{s3_prefix}/")

## Part 2: Create Model Package Group

In [ ]:
# Create Model Package Group
model_package_group_name = "timeseries-demand-forecasting"
model_package_group_description = "Time Series Demand Forecasting Models - NG Demand Forecasting"

try:
    sm_client.create_model_package_group(
        ModelPackageGroupName=model_package_group_name,
        ModelPackageGroupDescription=model_package_group_description
    )
    print(f"✓ Model Package Group '{model_package_group_name}' created successfully.")
except Exception as e:
    if "already exists" in str(e):
        print(f"✓ Model Package Group '{model_package_group_name}' already exists.")
    else:
        print(f"Error: {e}")

# Describe the group
response = sm_client.describe_model_package_group(
    ModelPackageGroupName=model_package_group_name
)
print(f"\nModel Package Group Status: {response['ModelPackageGroupStatus']}")

## Part 3: Create Model Packages for Each Model

In [ ]:
# Define model metadata for each model type
model_configs = {
    'Linear Regression': {
        'description': 'Linear Regression baseline model for time series demand forecasting',
        'algorithm': 'Linear Regression',
        'framework': 'scikit-learn',
        'content_types': ['text/csv', 'application/octet-stream'],
        'response_types': ['text/csv']
    },
    'Ridge Regression': {
        'description': 'Ridge Regression with L2 regularization for time series forecasting',
        'algorithm': 'Ridge Regression',
        'framework': 'scikit-learn',
        'content_types': ['text/csv', 'application/octet-stream'],
        'response_types': ['text/csv']
    },
    'Random Forest': {
        'description': 'Random Forest ensemble model with 200 trees',
        'algorithm': 'Random Forest',
        'framework': 'scikit-learn',
        'content_types': ['text/csv', 'application/octet-stream'],
        'response_types': ['text/csv']
    },
    'XGBoost': {
        'description': 'XGBoost gradient boosting model optimized for time series',
        'algorithm': 'XGBoost',
        'framework': 'XGBoost',
        'content_types': ['text/csv', 'application/octet-stream'],
        'response_types': ['text/csv']
    },
    'LightGBM': {
        'description': 'LightGBM light gradient boosting machine model',
        'algorithm': 'LightGBM',
        'framework': 'LightGBM',
        'content_types': ['text/csv', 'application/octet-stream'],
        'response_types': ['text/csv']
    },
    'ETS': {
        'description': 'Exponential Smoothing (Holt-Winters) for seasonal time series',
        'algorithm': 'Exponential Smoothing',
        'framework': 'statsmodels',
        'content_types': ['text/csv'],
        'response_types': ['text/csv']
    },
    'SARIMA': {
        'description': 'SARIMA(1,1,1)(1,1,1,12) statistical model with seasonal components',
        'algorithm': 'SARIMA',
        'framework': 'statsmodels',
        'content_types': ['text/csv'],
        'response_types': ['text/csv']
    }
}

# Get model performance data
performance_dict = {row['Model']: row for _, row in results_df.iterrows()}

# Create model packages
model_package_arns = {}

for model_name, config in model_configs.items():
    # For this example, we'll create metadata-only packages
    # In production, include full inference container specifications
    
    perf_data = performance_dict.get(model_name, {})
    
    # Format description as single line (AWS validation regex doesn't allow newlines)
    mae_val = f"{perf_data.get('MAE', 0):.2f}" if isinstance(perf_data.get('MAE'), (int, float)) else "N/A"
    rmse_val = f"{perf_data.get('RMSE', 0):.2f}" if isinstance(perf_data.get('RMSE'), (int, float)) else "N/A"
    r2_val = f"{perf_data.get('R2', 0):.4f}" if isinstance(perf_data.get('R2'), (int, float)) else "N/A"
    mape_val = f"{perf_data.get('MAPE', 0):.2f}" if isinstance(perf_data.get('MAPE'), (int, float)) else "N/A"
    
    description = f"{config['description']} | Performance: MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, MAPE={mape_val}%"
    
    try:
        response = sm_client.create_model_package(
            ModelPackageGroupName=model_package_group_name,
            ModelPackageDescription=description,
            ModelApprovalStatus='PendingManualApproval',
            InferenceSpecification={
                'Containers': [
                    {
                        'Image': 'placeholder-for-inference-image',  # In production, use actual image URI
                        'ModelDataUrl': model_metadata[model_name]['model_s3_path'],
                        'Framework': config['framework']
                    }
                ],
                'SupportedContentTypes': config['content_types'],
                'SupportedResponseMIMETypes': config['response_types']
            }
        )
        model_package_arn = response['ModelPackageArn']
        model_package_arns[model_name] = model_package_arn
        print(f"✓ Model Package created for {model_name}")
        print(f"  ARN: {model_package_arn}")
    except Exception as e:
        print(f"✗ Error creating package for {model_name}: {e}")

## Part 4: Create Model Cards with Detailed Metadata

In [ ]:
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# Create Model Cards for each model
for model_name, config in model_configs.items():
    perf_data = performance_dict.get(model_name, {})
    
    model_card_name = f"timeseries-fc-{model_name.replace(' ', '-').lower()}-{timestamp}"
    
    # Model Card content must follow AWS SageMaker's strict schema
    card_content = json.dumps({
        "model_overview": {
            "model_id": model_name,
            "model_name": f"Time Series {model_name}",
            "model_owner": "NG Demand Forecasting Team",
            "model_version": 1,
            "problem_type": "Time Series Forecasting",
            "algorithm_type": config['algorithm']
        },
        "intended_uses": {
            "purpose_of_model": "Forecast natural gas demand for the next 12 months",
            "intended_uses": "Production forecasting for demand planning and supply chain optimization",
            "factors_affecting_model_efficiency": "Model performance depends on data quality, seasonal patterns, and external factors like weather."
        },
        "business_details": {
            "business_problem": "Accurate demand forecasting for supply chain optimization",
            "business_stakeholders": "Operations, Supply Chain, and Finance teams"
        },
        "training_details": {
            "objective": "Minimize RMSE for time series demand forecasting"
        }
    })
    
    try:
        sm_client.create_model_card(
            ModelCardName=model_card_name,
            ModelCardStatus='Draft',
            Content=card_content
        )
        print(f"✓ Model Card created for {model_name}")
        print(f"  Card Name: {model_card_name}")
    except Exception as e:
        print(f"✗ Error creating model card for {model_name}: {e}")

## Part 5: Model Registry Summary

In [ ]:
# Create summary report
print("\n" + "="*80)
print("MODEL REGISTRY SUMMARY")
print("="*80)

print(f"\n📦 Model Package Group: {model_package_group_name}")
print(f"📍 S3 Location: s3://{bucket}/{s3_prefix}/")
print(f"⏰ Created: {timestamp}")

print("\n📋 REGISTERED MODELS:")
print("-" * 80)

for idx, row in results_df.iterrows():
    model_name = row['Model']
    rank = idx + 1
    rmse = row['RMSE']
    mape = row['MAPE']
    
    status = "🏆 BEST" if rank == 1 else f"  #{rank}"
    
    print(f"\n{status}: {model_name}")
    print(f"   ARN: {model_package_arns.get(model_name, 'N/A')}")
    print(f"   RMSE: {rmse:,.1f} | MAPE: {mape:.2f}% | R²: {row['R2']:.4f}")

In [ ]:
# Export model registry to CSV for tracking
registry_export = results_df.copy()
registry_export['ARN'] = registry_export['Model'].map(model_package_arns)
registry_export['Timestamp'] = timestamp
registry_export['S3_Path'] = registry_export['Model'].apply(
    lambda x: f"s3://{bucket}/{s3_prefix}/{x.replace(' ', '_').replace('(', '').replace(')', '')}.pkl"
)

registry_export.to_csv('model_registry.csv', index=False)
print("\n✓ Model registry exported to model_registry.csv")
print(registry_export[['Model', 'RMSE', 'MAPE', 'R2']].to_string())

## Part 6: Approve Best Model for Production

In [ ]:
# Get the best model's ARN
best_model_name = results_df.iloc[0]['Model']
best_model_arn = model_package_arns.get(best_model_name)

if best_model_arn:
    try:
        sm_client.update_model_package(
            ModelPackageArn=best_model_arn,
            ModelApprovalStatus='Approved'
        )
        print(f"✓ Best Model '{best_model_name}' approved for production")
        print(f"  ARN: {best_model_arn}")
    except Exception as e:
        print(f"Note: Could not update approval status: {e}")
        print(f"  Manual approval may be required in SageMaker console")

## Part 7: Query Model Registry

In [ ]:
# List all model packages in the group
print(f"\n📚 Querying Model Registry for: {model_package_group_name}\n")

paginator = sm_client.get_paginator('list_model_packages')
page_iterator = paginator.paginate(
    ModelPackageGroupName=model_package_group_name,
    PaginationConfig={'PageSize': 10}
)

model_package_list = []
for page in page_iterator:
    for package in page['ModelPackageSummaryList']:
        model_package_list.append({
            'Name': package['ModelPackageName'],
            'ARN': package['ModelPackageArn'],
            'Status': package['ModelPackageStatus'],
            'Approval Status': package.get('ModelApprovalStatus', 'Unknown'),
            'Created': package['CreationTime']
        })

print(f"Found {len(model_package_list)} model packages:\n")
for idx, pkg in enumerate(model_package_list, 1):
    print(f"{idx}. {pkg['Name']}")
    print(f"   Status: {pkg['Status']} | Approved: {pkg['Approval Status']}")
    print()